In [1]:
!/usr/src/tensorrt/bin/trtexec

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec
=== Model Options ===
  --uff=<file>                UFF model
  --onnx=<file>               ONNX model
  --model=<file>              Caffe model (default = no model, random weights used)
  --deploy=<file>             Caffe prototxt file
  --output=<name>[,<name>]*   Output names (it can be specified multiple times); at least one output is required for UFF and Caffe
  --uffInput=<name>,X,Y,Z     Input blob name and its dimensions (X,Y,Z=C,H,W), it can be specified multiple times; at least one is required for UFF models
  --uffNHWC                   Set if inputs are in the NHWC layout instead of NCHW (use X,Y,Z=H,W,C order in --uffInput)

=== Build Options ===
  --maxBatch                  Set max batch size and build an implicit batch engine (default = same size as --batch)
                              This option should not be used when the input model is ONNX or when dynamic shapes are provided.
  --minSh

In [2]:
import os

In [3]:
ONNX_MODEL_DIR = './Flask/Models/'

FLASK_MODEL_DIR = './Flask/Models/'

TRITON_MODEL_DIR = './Triton/Models/'
TRITON_CONFIG_FILE = 'config.pbtxt'
TRITON_MODEL_FILE = 'model.plan'

In [4]:
def save_tensorrt_model(onnx_model, trt_model, precision, config):
    trt_model = trt_model + '_' + precision
    onnx_model_path = os.path.join(ONNX_MODEL_DIR, onnx_model + '.onnx')
    flask_model_path = os.path.join(FLASK_MODEL_DIR, trt_model + '.plan')

    !sudo /usr/src/tensorrt/bin/trtexec \
        --onnx=$onnx_model_path \
        --explicitBatch \
        --$precision \
        --saveEngine=$flask_model_path

    triton_config_path = os.path.join(TRITON_MODEL_DIR, trt_model, TRITON_CONFIG_FILE)
    os.makedirs(os.path.dirname(triton_config_path), exist_ok=True)
    with open(triton_config_path, 'w') as f:
        f.write(config.strip())

    triton_model_path = os.path.join(TRITON_MODEL_DIR, trt_model, '1', TRITON_MODEL_FILE)
    os.makedirs(os.path.dirname(triton_model_path), exist_ok=True)
    !sudo cp $flask_model_path $triton_model_path
        
    print(flask_model_path)
    print(triton_config_path)
    print(triton_model_path)
    return

In [5]:
onnx_model = 'resnet34-onnx'
trt_model = 'resnet34-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/resnet34-onnx.onnx --explicitBatch --best --saveEngine=./Flask/Models/resnet34-trt_best.plan
[09/22/2022-21:23:15] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-21:23:15] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-21:23:15] [I] === Model Options ===
[09/22/2022-21:23:15] [I] Format: ONNX
[09/22/2022-21:23:15] [I] Model: ./Flask/Models/resnet34-onnx.onnx
[09/22/2022-21:23:15] [I] Output:
[09/22/2022-21:23:15] [I] === Build Options ===
[09/22/2022-21:23:15] [I] Max batch: explicit batch
[09/22/2022-21:23:15] [I] Workspace: 16 MiB
[09/22/2022-21:23:15] [I] minTiming: 1
[09/22/2022-21:23:15] [I] avgTiming: 8
[09/22/2022-21:23:15] [I] Precision: FP32+FP16+INT8
[09/22/2022-21:23:15] [I] Calibration: Dynamic
[09/22/2022-21:23:15] [I] Refit: Disabled
[09/22/2022-21:2

[09/22/2022-21:23:18] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +125, now: CPU 519, GPU 3202 (MiB)
[09/22/2022-21:23:20] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +241, GPU +247, now: CPU 760, GPU 3449 (MiB)
[09/22/2022-21:23:20] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[09/22/2022-21:23:36] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-21:26:56] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-21:26:57] [I] [TRT] Total Host Persistent Memory: 38848
[09/22/2022-21:26:57] [I] [TRT] Total Device Persistent Memory: 72795648
[09/22/2022-21:26:57] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-21:26:57] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 45 MiB, GPU 192 MiB
[09/22/2022-21:26:57] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown too

In [6]:
onnx_model = 'resnet34-onnx'
trt_model = 'resnet34-trt'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/resnet34-onnx.onnx --explicitBatch --fp16 --saveEngine=./Flask/Models/resnet34-trt_fp16.plan
[09/22/2022-21:27:02] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-21:27:02] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-21:27:02] [I] === Model Options ===
[09/22/2022-21:27:02] [I] Format: ONNX
[09/22/2022-21:27:02] [I] Model: ./Flask/Models/resnet34-onnx.onnx
[09/22/2022-21:27:02] [I] Output:
[09/22/2022-21:27:02] [I] === Build Options ===
[09/22/2022-21:27:02] [I] Max batch: explicit batch
[09/22/2022-21:27:02] [I] Workspace: 16 MiB
[09/22/2022-21:27:02] [I] minTiming: 1
[09/22/2022-21:27:02] [I] avgTiming: 8
[09/22/2022-21:27:02] [I] Precision: FP32+FP16
[09/22/2022-21:27:02] [I] Calibration: 
[09/22/2022-21:27:02] [I] Refit: Disabled
[09/22/2022-21:27:02] [I] Sp

[09/22/2022-21:27:20] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-21:28:44] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-21:28:45] [I] [TRT] Total Host Persistent Memory: 37312
[09/22/2022-21:28:45] [I] [TRT] Total Device Persistent Memory: 72795648
[09/22/2022-21:28:45] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-21:28:45] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 45 MiB, GPU 192 MiB
[09/22/2022-21:28:45] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took 3.05637ms to assign 3 blocks to 41 nodes requiring 2408448 bytes.
[09/22/2022-21:28:45] [I] [TRT] Total Activation Memory: 2408448
[09/22/2022-21:28:45] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +0, now: CPU 1049, GPU 3622 (MiB)
[09/22/2022-21:28:45] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +1, GPU +0, now: CPU 1050, GPU 36

In [7]:
onnx_model = 'resnet34-onnx'
trt_model = 'resnet34-trt'
precision = 'int8'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/resnet34-onnx.onnx --explicitBatch --int8 --saveEngine=./Flask/Models/resnet34-trt_int8.plan
[09/22/2022-21:28:52] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-21:28:52] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-21:28:52] [I] === Model Options ===
[09/22/2022-21:28:52] [I] Format: ONNX
[09/22/2022-21:28:52] [I] Model: ./Flask/Models/resnet34-onnx.onnx
[09/22/2022-21:28:52] [I] Output:
[09/22/2022-21:28:52] [I] === Build Options ===
[09/22/2022-21:28:52] [I] Max batch: explicit batch
[09/22/2022-21:28:52] [I] Workspace: 16 MiB
[09/22/2022-21:28:52] [I] minTiming: 1
[09/22/2022-21:28:52] [I] avgTiming: 8
[09/22/2022-21:28:52] [I] Precision: FP32+INT8
[09/22/2022-21:28:52] [I] Calibration: Dynamic
[09/22/2022-21:28:52] [I] Refit: Disabled
[09/22/2022-21:28:52]

[09/22/2022-21:28:55] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +126, now: CPU 519, GPU 3177 (MiB)
[09/22/2022-21:28:56] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +241, GPU +236, now: CPU 760, GPU 3413 (MiB)
[09/22/2022-21:28:56] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[09/22/2022-21:29:05] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-21:31:53] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-21:31:54] [I] [TRT] Total Host Persistent Memory: 37584
[09/22/2022-21:31:54] [I] [TRT] Total Device Persistent Memory: 140762112
[09/22/2022-21:31:54] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-21:31:54] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 9 MiB, GPU 384 MiB
[09/22/2022-21:31:54] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown too

In [8]:
onnx_model = 'mobilenet_v2-onnx'
trt_model = 'mobilenet_v2-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/mobilenet_v2-onnx.onnx --explicitBatch --best --saveEngine=./Flask/Models/mobilenet_v2-trt_best.plan
[09/22/2022-21:32:07] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-21:32:07] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-21:32:07] [I] === Model Options ===
[09/22/2022-21:32:07] [I] Format: ONNX
[09/22/2022-21:32:07] [I] Model: ./Flask/Models/mobilenet_v2-onnx.onnx
[09/22/2022-21:32:07] [I] Output:
[09/22/2022-21:32:07] [I] === Build Options ===
[09/22/2022-21:32:07] [I] Max batch: explicit batch
[09/22/2022-21:32:07] [I] Workspace: 16 MiB
[09/22/2022-21:32:07] [I] minTiming: 1
[09/22/2022-21:32:07] [I] avgTiming: 8
[09/22/2022-21:32:07] [I] Precision: FP32+FP16+INT8
[09/22/2022-21:32:07] [I] Calibration: Dynamic
[09/22/2022-21:32:07] [I] Refit: Disabled
[09/

[09/22/2022-21:32:10] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +145, now: CPU 450, GPU 3081 (MiB)
[09/22/2022-21:32:12] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +240, GPU +243, now: CPU 690, GPU 3324 (MiB)
[09/22/2022-21:32:12] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[09/22/2022-21:32:52] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-21:36:39] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-21:36:39] [I] [TRT] Total Host Persistent Memory: 103152
[09/22/2022-21:36:39] [I] [TRT] Total Device Persistent Memory: 4304896
[09/22/2022-21:36:39] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-21:36:39] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 8 MiB, GPU 32 MiB
[09/22/2022-21:36:39] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took 

In [9]:
onnx_model = 'mobilenet_v2-onnx'
trt_model = 'mobilenet_v2-trt'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/mobilenet_v2-onnx.onnx --explicitBatch --fp16 --saveEngine=./Flask/Models/mobilenet_v2-trt_fp16.plan
[09/22/2022-21:36:44] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-21:36:44] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-21:36:44] [I] === Model Options ===
[09/22/2022-21:36:44] [I] Format: ONNX
[09/22/2022-21:36:44] [I] Model: ./Flask/Models/mobilenet_v2-onnx.onnx
[09/22/2022-21:36:44] [I] Output:
[09/22/2022-21:36:44] [I] === Build Options ===
[09/22/2022-21:36:44] [I] Max batch: explicit batch
[09/22/2022-21:36:44] [I] Workspace: 16 MiB
[09/22/2022-21:36:44] [I] minTiming: 1
[09/22/2022-21:36:44] [I] avgTiming: 8
[09/22/2022-21:36:44] [I] Precision: FP32+FP16
[09/22/2022-21:36:44] [I] Calibration: 
[09/22/2022-21:36:44] [I] Refit: Disabled
[09/22/2022-21:3

[09/22/2022-21:36:47] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +145, now: CPU 450, GPU 3092 (MiB)
[09/22/2022-21:36:48] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +240, GPU +249, now: CPU 690, GPU 3341 (MiB)
[09/22/2022-21:36:48] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[09/22/2022-21:37:20] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-21:40:53] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-21:40:53] [I] [TRT] Total Host Persistent Memory: 110832
[09/22/2022-21:40:53] [I] [TRT] Total Device Persistent Memory: 4304896
[09/22/2022-21:40:53] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-21:40:53] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 8 MiB, GPU 32 MiB
[09/22/2022-21:40:53] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took 

In [10]:
onnx_model = 'mobilenet_v2-onnx'
trt_model = 'mobilenet_v2-trt'
precision = 'int8'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/mobilenet_v2-onnx.onnx --explicitBatch --int8 --saveEngine=./Flask/Models/mobilenet_v2-trt_int8.plan
[09/22/2022-21:40:57] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-21:40:57] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-21:40:57] [I] === Model Options ===
[09/22/2022-21:40:57] [I] Format: ONNX
[09/22/2022-21:40:57] [I] Model: ./Flask/Models/mobilenet_v2-onnx.onnx
[09/22/2022-21:40:57] [I] Output:
[09/22/2022-21:40:57] [I] === Build Options ===
[09/22/2022-21:40:57] [I] Max batch: explicit batch
[09/22/2022-21:40:57] [I] Workspace: 16 MiB
[09/22/2022-21:40:57] [I] minTiming: 1
[09/22/2022-21:40:57] [I] avgTiming: 8
[09/22/2022-21:40:57] [I] Precision: FP32+INT8
[09/22/2022-21:40:57] [I] Calibration: Dynamic
[09/22/2022-21:40:57] [I] Refit: Disabled
[09/22/20

[09/22/2022-21:41:01] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +150, now: CPU 450, GPU 3097 (MiB)
[09/22/2022-21:41:02] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +240, GPU +237, now: CPU 690, GPU 3334 (MiB)
[09/22/2022-21:41:02] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[09/22/2022-21:41:25] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-21:44:05] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-21:44:05] [I] [TRT] Total Host Persistent Memory: 107440
[09/22/2022-21:44:05] [I] [TRT] Total Device Persistent Memory: 7301120
[09/22/2022-21:44:05] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-21:44:05] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 8 MiB, GPU 32 MiB
[09/22/2022-21:44:05] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took 

In [11]:
onnx_model = 'efficientnet_b0-onnx'
trt_model = 'efficientnet_b0-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/efficientnet_b0-onnx.onnx --explicitBatch --best --saveEngine=./Flask/Models/efficientnet_b0-trt_best.plan
[09/22/2022-21:44:10] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-21:44:10] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-21:44:10] [I] === Model Options ===
[09/22/2022-21:44:10] [I] Format: ONNX
[09/22/2022-21:44:10] [I] Model: ./Flask/Models/efficientnet_b0-onnx.onnx
[09/22/2022-21:44:10] [I] Output:
[09/22/2022-21:44:10] [I] === Build Options ===
[09/22/2022-21:44:10] [I] Max batch: explicit batch
[09/22/2022-21:44:10] [I] Workspace: 16 MiB
[09/22/2022-21:44:10] [I] minTiming: 1
[09/22/2022-21:44:10] [I] avgTiming: 8
[09/22/2022-21:44:10] [I] Precision: FP32+FP16+INT8
[09/22/2022-21:44:10] [I] Calibration: Dynamic
[09/22/2022-21:44:10] [I] Refit: Disa

[09/22/2022-21:44:14] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +139, now: CPU 457, GPU 3201 (MiB)
[09/22/2022-21:44:15] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +241, GPU +250, now: CPU 698, GPU 3451 (MiB)
[09/22/2022-21:44:15] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[09/22/2022-21:48:41] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-21:55:19] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-21:55:20] [I] [TRT] Total Host Persistent Memory: 157264
[09/22/2022-21:55:20] [I] [TRT] Total Device Persistent Memory: 5713920
[09/22/2022-21:55:20] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-21:55:20] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 12 MiB, GPU 51 MiB
[09/22/2022-21:55:20] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took

In [12]:
onnx_model = 'efficientnet_b0-onnx'
trt_model = 'efficientnet_b0-trt'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/efficientnet_b0-onnx.onnx --explicitBatch --fp16 --saveEngine=./Flask/Models/efficientnet_b0-trt_fp16.plan
[09/22/2022-21:55:25] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-21:55:25] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-21:55:25] [I] === Model Options ===
[09/22/2022-21:55:25] [I] Format: ONNX
[09/22/2022-21:55:25] [I] Model: ./Flask/Models/efficientnet_b0-onnx.onnx
[09/22/2022-21:55:25] [I] Output:
[09/22/2022-21:55:25] [I] === Build Options ===
[09/22/2022-21:55:25] [I] Max batch: explicit batch
[09/22/2022-21:55:25] [I] Workspace: 16 MiB
[09/22/2022-21:55:25] [I] minTiming: 1
[09/22/2022-21:55:25] [I] avgTiming: 8
[09/22/2022-21:55:25] [I] Precision: FP32+FP16
[09/22/2022-21:55:25] [I] Calibration: 
[09/22/2022-21:55:25] [I] Refit: Disabled
[09/22/

[09/22/2022-21:55:29] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +139, now: CPU 457, GPU 3200 (MiB)
[09/22/2022-21:55:30] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +241, GPU +247, now: CPU 698, GPU 3447 (MiB)
[09/22/2022-21:55:30] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[09/22/2022-21:58:02] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-22:03:36] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-22:03:37] [I] [TRT] Total Host Persistent Memory: 157184
[09/22/2022-22:03:37] [I] [TRT] Total Device Persistent Memory: 5713920
[09/22/2022-22:03:37] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-22:03:37] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 12 MiB, GPU 51 MiB
[09/22/2022-22:03:37] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took

In [13]:
onnx_model = 'efficientnet_b0-onnx'
trt_model = 'efficientnet_b0-trt'
precision = 'int8'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=./Flask/Models/efficientnet_b0-onnx.onnx --explicitBatch --int8 --saveEngine=./Flask/Models/efficientnet_b0-trt_int8.plan
[09/22/2022-22:03:42] [W] --explicitBatch flag has been deprecated and has no effect!
[09/22/2022-22:03:42] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[09/22/2022-22:03:42] [I] === Model Options ===
[09/22/2022-22:03:42] [I] Format: ONNX
[09/22/2022-22:03:42] [I] Model: ./Flask/Models/efficientnet_b0-onnx.onnx
[09/22/2022-22:03:42] [I] Output:
[09/22/2022-22:03:42] [I] === Build Options ===
[09/22/2022-22:03:42] [I] Max batch: explicit batch
[09/22/2022-22:03:42] [I] Workspace: 16 MiB
[09/22/2022-22:03:42] [I] minTiming: 1
[09/22/2022-22:03:42] [I] avgTiming: 8
[09/22/2022-22:03:42] [I] Precision: FP32+INT8
[09/22/2022-22:03:42] [I] Calibration: Dynamic
[09/22/2022-22:03:42] [I] Refit: Disabled


[09/22/2022-22:03:45] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +159, GPU +139, now: CPU 458, GPU 3186 (MiB)
[09/22/2022-22:03:47] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +240, GPU +246, now: CPU 698, GPU 3432 (MiB)
[09/22/2022-22:03:47] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[09/22/2022-22:06:33] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[09/22/2022-22:11:15] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[09/22/2022-22:11:15] [I] [TRT] Total Host Persistent Memory: 132320
[09/22/2022-22:11:15] [I] [TRT] Total Device Persistent Memory: 10992128
[09/22/2022-22:11:15] [I] [TRT] Total Scratch Memory: 0
[09/22/2022-22:11:15] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 9 MiB, GPU 51 MiB
[09/22/2022-22:11:15] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took